# NAC coloring search

In this notebook we provide utilities to run benchmarks, analyze results and experiment with our code.
As the package is still in development, not all parts must be user-friendly, still we try to provide good enough abstraction.

First we define some utility functions for Matplotlib,
then we provide utility functions for loading and storing benchmarks results,
after we provide a framework for loading graph classes, defining benchmarks and running them.
Lastly, we provide tools for quick results analyzations.

Many utility functions were moved from the notebook
into a separate file to improve clarity, see `benchmarks/notebook_utils.py`.

For some advanced functionality in datasets and running PyTest it is necessary to PyRigi v0.3.0.
Either install it using `pip install pyrigi==0.3.0` or clone it into the project root directory from https://github.com/PyRigi/PyRigi/tree/0.3.0.
The `nac` package itself is completely independent.

In [ ]:
from typing import *
from dataclasses import dataclass
from collections import defaultdict, deque
import random
import importlib
from random import Random
from enum import Enum

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline
from matplotlib.backends import backend_agg
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator

import numpy as np
import pandas as pd
import networkx as nx
import os
import time
import datetime
import signal
import itertools
import base64

from tqdm import tqdm

import nac as nac
import nac.util
from nac import MonochromaticClassType
importlib.reload(nac)
importlib.reload(nac.util)

import benchmarks
from benchmarks import dataset
import benchmarks.notebook_utils
from benchmarks.notebook_utils import *
importlib.reload(benchmarks)
importlib.reload(dataset)
importlib.reload(benchmarks.notebook_utils)

seed=42

### Benchmarks directory

You can either choose to use our precomputed results or run the benchmarks yourself.
The algorithms take usually tens or hunderes of miliseconds to run,
but there is plenty of graphs and strategies combinations, so times add up.

In [ ]:
OUTPUT_DIR_PRECOMPUTED = os.path.join("benchmarks", "precomputed")
OUTPUT_DIR_LOCAL = os.path.join("benchmarks", "local")

benchmarks.notebook_utils.OUTPUT_DIR = OUTPUT_DIR_PRECOMPUTED
os.makedirs(benchmarks.notebook_utils.OUTPUT_DIR, exist_ok=True)

# Loading graph classes

In this section we load graphs that can be later used for running benchmarks.
The graphs are not in any specified order and they differ in size ranges.
Graphs are stored in the `graph6` format in the `graphs_store` directory.

In [ ]:
class Graphs:
    """
    Randomly generated minimally rigid (Laman) graphs of various sizes
    """
    minimally_rigid_random = LazyList(lambda: dataset.load_laman_random_graphs())
    """
    Graphs with no 3 nor 4 cycles up to 42 vertices
    """
    no_3_nor_4_cycles = LazyList(lambda: dataset.load_no_3_nor_4_cycle_graphs())
    """
    Graps generated according to yet unpublished formulla that garanties that these graphs should either have no or small number of NAC-colorings
    """
    sparse_with_few_colorings = LazyList(lambda: dataset.load_sparse_with_few_colorings_graphs())
    """
    Globally rigid graphs
    """
    globally_rigid = LazyList(lambda: dataset.load_globally_rigid_graphs())

    """
    Loads all the minimally rigid (Laman) graphs of the given size, pregenerated files allow the range of [5, 11]
    """
    def load_all_minimally_rigid(vertex_no: int) -> List[nx.Graph]:
        return list(dataset.load_laman_all(vertices_no=vertex_no))

# Running on all minimally rigid graphs

This is a function that can be used for benchmarking of finding all the NAC-colorings of some graph class.
This function can provide only total times, not timer per graph. That is job of the following benchmarks.

In [ ]:
def benchmarks_all_NAC_coloring_on_class(
    graphs: List[nx.Graph],
    rounds: int,
    strategy: str,
    use_monochromatic_classes: bool,
    use_has_coloring_check: bool = False,
) -> float:
    start = time.time()
    monochromatic_class_type=nac.MonochromaticClassType.MONOCHROMATIC if use_monochromatic_classes else nac.MonochromaticClassType.TRIANGLES

    for _ in range(rounds):
        for graph in graphs:
            # the fastest way to collect an iterable in Python
            deque(nac.NAC_colorings(
                graph,
                relabel_strategy="none",
                algorithm=strategy,
                monochromatic_class_type=monochromatic_class_type,
                use_has_coloring_check=use_has_coloring_check,
            ), 0)

    return (time.time() - start) / rounds

def benchmarks_all_NAC_coloring_minimally_rigid(
    vertex_no: int,
    rounds: int,
):
    graphs = list(Graphs.load_all_minimally_rigid(vertex_no))
    for strategy, use_monochromatic_classes in [
        ('naive', False),
        ('naive', True),
        ('cycles', True),
        ('subgraphs-linear-neighbors_degree-4', True),
    ]:
        print(f"[{vertex_no:2}]: # {strategy} (monochrom: {use_monochromatic_classes})")
        runtime = benchmarks_all_NAC_coloring_on_class(
            graphs=graphs,
            rounds=rounds,
            strategy=strategy,
            use_monochromatic_classes=use_monochromatic_classes,
        )
        print(f"[{vertex_no:2}]: > {runtime:.3f} s")

if False:
    for n in range(5, 11+1):
        benchmarks_all_NAC_coloring_minimally_rigid(
            vertex_no=n,
            rounds=3 if n <= 10 else 1,
        )

# Storing and loading benchmark results

Each row represents performance of a graph with a given strategy.
The difference between the first and all variant is that
in the all variants we search for all NAC-colorings,
but in the first variant we search only.

The export CSV columns are:
- `graph` - base64 encoded bytes of graph6 encoded graph
- `dataset` - class of the graph, `minimally_ridig_random`, `no_3_nor_4_cycles`, `globally_rigid`, ...
- `vertex_no` - the number of vertices of the graph
- `edge_no` - the number of edges of the graph
- `triangle_components_no` - the number of triangle components of the graph
- `monochromatic_classes_no` - the number of monochromatic classes of the graph
- `relabel` - relabel strategy (relabels vertices before the main algorithm is run, here we have only `none` or `random`)
- `split` - splitting strategy
- `merge` - merging strategy
- `subgraph_size` - the target initial size of subgraphs in monochromatic components
- `used_monochromatic_classes` - if monochromatic classes were used to run the test, `False` means triangle components were used
- `nac_any_finished` - if any of the tests finished in time
- `nac_{first|all}_coloring_no` - the number of NAC-colorings of the graph, for the first variant limited to 1
- `nac_{first|all}_mean_time` - the time required to find first/all NAC-colorings in milliseconds
- `nac_{first|all}_rounds` - the number of rounds used to run the benchmarks
- `nac_{first|all}_check_cycle_mask` - the number of cycle mask checks performed
- `nac_{first|all}_check_is_NAC` - the number of `IsNACColorng` checks performed

In [ ]:
display(COLUMNS)

# Strategies

The interface of the NAC-coloring search function looks like this:
```python
def NAC_colorings(
    graph: nx.Graph,
    algorithm: str = "subgraphs",
    relabel_strategy: str = "none",
    monochromatic_class_type: MonochromaticClassType = MonochromaticClassType.MONOCHROMATIC,
    use_decompositions: bool = True,
    use_has_coloring_check: bool = True,
    seed: int | None = None,
) -> Iterable[NACColoring]:
```

The relabel strategy is either `"none"` or `"random"`.
`MonochromaticClassType` types are `MONOCHROMATIC` that creates monochromatic classes as described in the paper,
`TRIANGLES` that finds only triangle connected components and
`EDGES` that uses no monochromatic classes optimization.
The `use_decompositions` switch is responsible for enabling checks for articulation points and related decomposition into blocks.
The `use_has_coloring_check` runs some polynomial checks if a NAC-coloring can exist. If not, the whole search is skipped.
`seed` is used by strategies internally as only pseudo random number generators are used.

The most important field is the `algorithm` field.
Possible values are:
- `"naive"` - runs naive algorithm
- `"cycles"` - runs naive algorithm improved by cycles detection
- `"subgraphs"` - runs so far optimal algorithm for larger graphs based on subgraph decomposition
- `"subgraphs-{merge_strategy}-{split_strategy}-{size_of_subgraphs}"` - runs the specified strategy combination with subgraph decomposition

In our code strategies are represented as four-tupples.

In [ ]:
class Promising:
    RELABELING = [
        "none",
        "random",
    ]
    SPLITTING = [
        "none",
        "neighbors",
        "neighbors_degree",
    ]
    MERGE = [
        "linear",
        "shared_vertices",
    ]
    # This is not optimal for every graph, but does not hurt performance significantly
    SIZES = [6]

    strategies = list(itertools.product(
        RELABELING, SPLITTING, MERGE, SIZES,
    ))
print(f"Strategies: {len(Promising.strategies)}")

In [ ]:
def create_subgraph_strategy(param: Tuple[str, str, str, int]) -> Tuple[str, str]:
    relabel, split, merge, subgraph = param
    algo_name = "subgraphs-{}-{}-{}".format( merge, split, subgraph)
    return (relabel, algo_name)

In case you want to play with the notebook we predefined some strategies as example.

In [ ]:
STRATEGY_NONE_LINEAR = create_subgraph_strategy(("", "none", "linear", 6))[1]
STRATEGY_NEIGHBORS_LINEAR = create_subgraph_strategy(("", "neighbors", "linear", 6))[1]
STRATEGY_NEIGHBORS_DEGREE_LINEAR = create_subgraph_strategy(("", "neighbors_degree", "linear", 6))[1]

list(nac.NAC_colorings(
    graph=nx.path_graph(4),
    algorithm=STRATEGY_NEIGHBORS_DEGREE_LINEAR,
))

### Running and recording benchmarks

This cell serves as the main interface for running benchmarks, see the function's doc string.
It is compatible only with subgraph decomposition based strategies, for the naive search other functions have to be written.

In [ ]:
def measure_for_graph_class(
    dataset_name: str,
    graphs: Iterable[nx.Graph],
    all_max_vertex_no: int,
    rounds:int,
    graph_timeout: int,
    use_monochromatic_classes: bool = True,
    df_seen: pd.DataFrame | None = load_records(),
    save_every: int | None = 5*60,
) -> pd.DataFrame:
    """
    Runs benchmarks for the given graph class.

    Parameters:
        dataset_name: Name of the dataset stored in the output csv
        graphs: Iterable of graphs to benchmark
        all_max_vertex_no: Maximum vertex number to search for all NAC-colorings
        rounds: Number of rounds to run for each graph
        graph_timeout: Timeout for each graph in seconds
        use_monochromatic_classes: Whether to use monochromatic classes or tiriangle connected components
        df_seen: Dataframe with already measured data, so already tried graphs and strategies can be skipped
        save_every: save progress every number of seconds
    """

    dataset_name = dataset_name.replace(" ", "_").lower()
    if df_seen is None:
        df_seen = toBenchmarkResults()
    df_seen = df_seen.query(f"dataset == '{dataset_name}'")

    results: List[MeasurementResult] = []
    all_results: List[MeasurementResult] = []

    last_save = time.time()

    for graph in tqdm(graphs):
        # this would be a functin if python would not have broken scoping
        if save_every is not None:
            now = time.time()
            if now - last_save > save_every:
                all_results.extend(results)
                df = toBenchmarkResults(results)
                update_stored_data([df], head_loaded=False)
                results = []
                last_save = now


        all_colorings = all_max_vertex_no >= graph.number_of_nodes()
        trianlge_classes = len(nac.find_monochromatic_classes(graph=graph, class_type=MonochromaticClassType.TRIANGLES)[1])
        monochromatic_classes = len(nac.find_monochromatic_classes(graph=graph, class_type=MonochromaticClassType.MONOCHROMATIC)[1])

        graph_id = graph_to_id(graph)
        df_graph = df_seen.query(f"graph == '{graph_id}'")

        for strategy in Promising.strategies:
            # skip test that already run
            prev_record = df_graph.query(
                f"relabel == '{strategy[0]}'"
                + f" and split == '{strategy[1]}'"
                + f" and merging == '{strategy[2]}'"
                + f" and subgraph_size == {strategy[3]}"
                + f" and used_monochromatic_classes == {use_monochromatic_classes}"
            )
            if len(prev_record) > 0:
                if graph.number_of_nodes() > all_max_vertex_no or list(prev_record["nac_all_mean_time"])[-1] > 0:
                    continue

            try:
                search_res = nac_benchmark_core(
                    graph,
                    rounds=rounds,
                    first_only=not all_colorings,
                    strategy=create_subgraph_strategy(strategy),
                    use_monochromatic_classes=use_monochromatic_classes,
                    time_limit=graph_timeout,
                )

                relabel, split, merge, subgraph_size = strategy
                res = create_measurement_result(
                    graph=graph,
                    dataset_name=dataset_name,
                    trianlge_classes=trianlge_classes,
                    monochromatic_classes=monochromatic_classes,
                    nac_first=search_res.first,
                    nac_all=search_res.all,
                    relabel_strategy=relabel,
                    split_strategy=split,
                    merge_strategy=merge,
                    subgraph_size=subgraph_size,
                    used_monochromatic_classes=use_monochromatic_classes,
                )
                results.append(res)
            except Exception as e:
                print("Exception:", e)

    all_results.extend(results)
    df = toBenchmarkResults(results)
    update_stored_data([df], head_loaded=False)

    df = toBenchmarkResults(all_results)
    df = df.sort_values(by=["nac_all_mean_time", "nac_first_mean_time"])
    return df

# Running benchmarks

You can run any of these cells by changing the condition and running the cell. As described above, benchmarks take long to run because each graph is run 3 times for each enabled strategy.
Each run on a graph takes tens or hundreds of milliseconds to run and that adds up.
There is autosave enabled that stores progress every 5 minutes.
Do not forget to change `OUTPUT_DIR` at the beginning of the notebook otherwise the tests will be skipped as they are already precomputed for you.

### Minimally rigid - Random

In [ ]:
if False:
    measure_for_graph_class(
        "Minimally rigid random",
        Graphs.minimally_rigid_random,
        all_max_vertex_no=16,
        rounds=3,
        graph_timeout=3,
    )

### No 3 nor 4 cycles

In [ ]:
if False:
    df_no_3_nor_4_cycles = measure_for_graph_class(
        "No 3 nor 4 cycles",
        Graphs.no_3_nor_4_cycles,
        all_max_vertex_no=0,
        rounds=3,
        graph_timeout=3,
    )

### Sparse with few NAC-colorings

In [ ]:
if False:
    measure_for_graph_class(
        "few_colorings",
        Graphs.sparse_with_few_colorings,
        all_max_vertex_no=16,
        rounds=3,
        graph_timeout=3,
    )

### Globally rigid

In [ ]:
if False:
    measure_for_graph_class(
        "globally_rigid",
        [g for g in Graphs.globally_rigid if g.number_of_nodes() < 17],
        all_max_vertex_no=16,
        rounds=3,
        graph_timeout=3,
    )

# Analytics

In this section we provide a framework for plotting results of the previous benchmarks.

**All the chars plotted bellow in this section are created from runs with more than one monochromatic classes.**
If that is the case, the results can be obtained immediately as the answer is trivial.
Therefore, we filter them out.

The first group of graphs show the time required to find
a first/all NAC coloring based on the number of vertices or the number of monochromatic classes.
In one row you can see mean and median plots with lines for each strategy.
Graphs show mean and median, but it is not hard to add additional aggregation function to the framework.

In [ ]:
df_analytics = load_records()

df_analytics = df_analytics.query("nac_any_finished == True")
df_analytics = df_analytics.assign(split_merging=lambda x: x["split"] + " & " + x["merging"])

df_analytics = df_analytics.query("monochromatic_classes_no > 1")

# display(df_analytics.info())
print("Records:", df_analytics.shape[0], "graphs:", df_analytics["graph"].nunique())
display(df_analytics.columns)
display(list(df_analytics["dataset"].unique()))
display(list(df_analytics["relabel"].unique()))
display(list(df_analytics["split"].unique()))
display(list(df_analytics["merging"].unique()))

In [ ]:
if True:
    [display(fig) for fig in plot_frame("Minimally rigid - Random", df_analytics.query("dataset == 'minimally_rigid_random'"))]

In [ ]:
if True:
    [display(fig) for fig in plot_frame("No 3 nor 4 cycles", df_analytics.query("dataset == 'no_3_nor_4_cycles'"))]

In [ ]:
if True:
    [display(fig) for fig in plot_frame("Sparse with few colorings - None", df_analytics.query("dataset == 'few_colorings'"))]

In [ ]:
if True:
    [display(fig) for fig in plot_frame("Globally rigid", df_analytics.query("dataset == 'globally_rigid'"))]

## The number of checks needed

This group of graphs compares the number of checks performed by our algorithm and by naive algorithm
using either no monochromatic classes, triangle components or monochromatic classes described in the article.

Unless you change anything, the result is plotted from the whole benchmarks dataset - all the graphs classes are used.
You can add `query("dataset == '...'")` to show the graph for a specific dataset.

The number of `IsNACColoring` checks called compared to
the naive approach without or with triangle/monochromatic classes.

It is expected that the number of `IsNACColoring` checks will be smaller than the `CycleMask` checks as the `CycleMask` checks happen every time, but `IsNACColoring` checks happen only if the previous checks fail.

In [ ]:
if True:
    [display(fig) for fig in plot_is_NAC_coloring_calls(df_analytics)]